In [1]:
import numpy as np

In [8]:
class Jaya():
    def __init__(self):
        self.obj = None
        self.population = None
        self.dimension = None
        self.ub = None
        self.lb = None
        self.optimization = "maximize"
        self.max_iter = None
        self.global_best_agent = None
        
    def compile(self,obj,pop_size,dimension,lb,ub,optimization="maximize"):
        self.obj = obj
        self.population_size = pop_size
        self.dimension = dimension
        self.ub = ub
        self.lb = lb
        self.optimization=optimization
        
    def __initialize(self):
        self.population = np.clip(np.random.rand(self.population_size,self.dimension),self.lb,self.ub)
        self.ev_value = np.empty(self.population_size)
        
        for i,value in enumerate(self.population):
            self.ev_value[i] = self.obj(value)
        
    def __update_best_worst(self):
        max_id = np.argmax(self.ev_value)
        min_id = np.argmin(self.ev_value)
        
        if self.optimization == "maximize":
            self.best_agent = (self.population[max_id],self.ev_value[max_id])
            self.worst_agent = (self.population[min_id],self.ev_value[min_id])
        elif self.optimization == "minimize":
            self.worst_agent = (self.population[max_id],self.ev_value[max_id])
            self.best_agent = (self.population[min_id],self.ev_value[min_id])
            
    def __update_global_best(self):
        if self.global_best_agent == None:
            self.global_best_agent = self.best_agent
        elif self.optimization == "maximize" and self.best_agent[1] > self.global_best_agent[1]:
            self.global_best_agent = self.best_agent
        elif self.optimization == "minimize" and self.best_agent[1] < self.global_best_agent[1]:
            self.global_best_agent = self.best_agent
           
    def train(self,max_iter):
        
        self.__initialize()
        self.__update_best_worst()
        for i in range(max_iter):
            
            self.__update_best_worst()
            self.__update_global_best()
            r1 = np.random.rand(self.population_size,self.dimension)
            r2 = np.random.rand(self.population_size,self.dimension)
            population_updated = self.population + r1 * (self.best_agent[0] - np.abs(self.population)) - r2 * (self.worst_agent[0] - np.abs(self.population))
            
            
            
            self.population = np.clip(population_updated,lb,ub)
            
            for i,value in enumerate(self.population):
                self.ev_value[i] = self.obj(value)     
        return self.global_best_agent
            

In [9]:
from benchmarking.benchmark import *
function=[Sphere,Schwefel_2_22,Schwefel_1_2,Rosenbrock,Step,Quartic_with_noise,Ackley,SumSquare]


In [7]:
for i in function:
    func=i()
    values=np.empty(100)
    for l in range(100):
        lb = np.array([func.range[0]]).repeat(func.dim)
        ub = np.array([func.range[1]]).repeat(func.dim)
        jaya = Jaya()
        jaya.compile(func.get_algorithm(),pop_size=100,dimension=func.dim,lb=lb,ub=ub,optimization = "minimize")
        values[l]=jaya.train(100)[-1]
    print(f"{func.name}\t ==> Min = {np.min(values)} Max = {np.max(values)} Mean = {np.mean(values)}")

Sphere	 ==> Min = 41663.96113783932 Max = 66717.6917913425 Mean = 55284.130751138546
Schwefel_2_22	 ==> Min = -9.999999999999999e+29 Max = -1.1155386146585538e+29 Mean = -5.3545208954595605e+29
Schwefel_1_2	 ==> Min = 662669.6768064382 Max = 1340511.7899881355 Mean = 1079752.3276382936
Rosenbrock	 ==> Min = 74831594.71175182 Max = 252283657.2182821 Mean = 179501755.54120886
Step	 ==> Min = 37807.04764719771 Max = 65680.4502439344 Mean = 54716.209449727015
Quartic_with_noise	 ==> Min = 1766637589.0963202 Max = 5152593573.628827 Mean = 3255741983.269941
Ackley	 ==> Min = 19.678119137505508 Max = 20.076964064015687 Mean = 19.962966994614888
SumSquare	 ==> Min = 4334.140522216574 Max = 9329.317803266262 Mean = 7603.140717286845
